In [13]:
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms

data = ImageFolder(root = "./image", transform=transforms.Compose(
    [transforms.ToTensor(),
     transforms.Resize((50,50))]))

In [14]:
data.classes, data.class_to_idx, len(data.imgs)

(['anya', 'bekki'], {'anya': 0, 'bekki': 1}, 196)

In [15]:
from torch.utils.data import DataLoader

data_loader = DataLoader(data, batch_size=8, shuffle=True)

In [16]:
for i,j in data_loader:
    print(i.shape, j) 

torch.Size([8, 3, 50, 50]) tensor([0, 1, 0, 0, 0, 0, 0, 0])
torch.Size([8, 3, 50, 50]) tensor([0, 1, 1, 0, 1, 1, 0, 0])
torch.Size([8, 3, 50, 50]) tensor([0, 1, 0, 1, 0, 1, 1, 1])
torch.Size([8, 3, 50, 50]) tensor([1, 0, 1, 1, 1, 0, 1, 1])
torch.Size([8, 3, 50, 50]) tensor([0, 0, 1, 1, 0, 0, 1, 0])
torch.Size([8, 3, 50, 50]) tensor([1, 0, 1, 1, 0, 0, 0, 0])
torch.Size([8, 3, 50, 50]) tensor([0, 0, 1, 0, 0, 0, 0, 0])
torch.Size([8, 3, 50, 50]) tensor([1, 1, 1, 0, 1, 0, 0, 1])
torch.Size([8, 3, 50, 50]) tensor([1, 0, 1, 1, 0, 0, 1, 0])
torch.Size([8, 3, 50, 50]) tensor([0, 0, 0, 0, 0, 0, 0, 0])
torch.Size([8, 3, 50, 50]) tensor([0, 1, 1, 0, 0, 1, 0, 0])
torch.Size([8, 3, 50, 50]) tensor([0, 1, 1, 1, 0, 0, 0, 0])
torch.Size([8, 3, 50, 50]) tensor([1, 0, 0, 0, 0, 1, 0, 0])
torch.Size([8, 3, 50, 50]) tensor([1, 1, 0, 0, 0, 1, 0, 0])
torch.Size([8, 3, 50, 50]) tensor([1, 0, 0, 1, 0, 1, 0, 0])
torch.Size([8, 3, 50, 50]) tensor([1, 1, 0, 1, 1, 1, 1, 0])
torch.Size([8, 3, 50, 50]) tensor([0, 1,

In [17]:
# CNN 모델 클래스 생성
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 9 * 9, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 9 * 9)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [18]:
# 모델 객체 생성, Adam 옵티마이저 생성
import torch.optim as optim

model = CNN()
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [21]:
import torchmetrics.functional as metrics

for epoch in range(10):
    for i, data in enumerate(data_loader):
        x, y = data
        optimizer.zero_grad()
        output = model(x)
        loss = criterion(output, y)
        loss.backward()
        optimizer.step()
        if i % 10 == 0:
            print(epoch, i, loss.item())
            
    # 트레인 정확도, f1 계산
    y_pred = model(x)
    print(metrics.accuracy(y_pred, y))
    print(metrics.f1(y_pred, y))

0 0 0.24083487689495087
0 10 0.10990116745233536
0 20 0.45305803418159485


TypeError: accuracy() missing 1 required positional argument: 'task'